### Lab | Random Forests

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [2]:
churnData= pd.read_csv('Customer-Churn.csv')

In [18]:
churnData.isna().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [19]:
churnData=churnData.dropna(subset = ['TotalCharges'])
churnData.isna().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [20]:
churnData['TotalCharges']= pd.to_numeric(churnData['TotalCharges'], errors = 'coerce' ) # We need to convert TotalCharges to numerical as it is not categorical.

In [21]:
churnData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7032 non-null   object 
 1   SeniorCitizen     7032 non-null   int64  
 2   Partner           7032 non-null   object 
 3   Dependents        7032 non-null   object 
 4   tenure            7032 non-null   int64  
 5   PhoneService      7032 non-null   object 
 6   OnlineSecurity    7032 non-null   object 
 7   OnlineBackup      7032 non-null   object 
 8   DeviceProtection  7032 non-null   object 
 9   TechSupport       7032 non-null   object 
 10  StreamingTV       7032 non-null   object 
 11  StreamingMovies   7032 non-null   object 
 12  Contract          7032 non-null   object 
 13  MonthlyCharges    7032 non-null   float64
 14  TotalCharges      7032 non-null   float64
 15  Churn             7032 non-null   object 
dtypes: float64(2), int64(2), object(12)
memory

In [7]:
### Let's divide the dataset into 3 groups. Categorical,numerical and target.

In [22]:
X = churnData.drop(['Churn'], axis=1)
X

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.50
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.50
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.90
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.60


In [23]:
X_num= X.select_dtypes(include='number')
X_cat= X.select_dtypes(include='object')

In [24]:
X_num

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0,0,1,29.85,29.85
1,0,34,56.95,1889.50
2,0,2,53.85,108.15
3,0,45,42.30,1840.75
4,0,2,70.70,151.65
...,...,...,...,...
7038,0,24,84.80,1990.50
7039,0,72,103.20,7362.90
7040,0,11,29.60,346.45
7041,1,4,74.40,306.60


In [25]:
X_cat

,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract
0,Female,Yes,No,No,No,Yes,No,No,No,No,Month-to-month
1,Male,No,No,Yes,Yes,No,Yes,No,No,No,One year
2,Male,No,No,Yes,Yes,Yes,No,No,No,No,Month-to-month
3,Male,No,No,No,Yes,No,Yes,Yes,No,No,One year
4,Female,No,No,Yes,No,No,No,No,No,No,Month-to-month
...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,One year
7039,Female,Yes,Yes,Yes,No,Yes,Yes,No,Yes,Yes,One year
7040,Female,Yes,Yes,No,Yes,No,No,No,No,No,Month-to-month
7041,Male,Yes,No,Yes,No,No,No,No,No,No,Month-to-month


In [26]:
y=churnData[['Churn']]
y

,Churn
0,No
1,No
2,Yes
3,No
4,Yes
...,...
7038,No
7039,No
7040,No
7041,Yes


In [13]:
## Now its time to encode categoricals. We have 2 options, label encoder or one hot encoding. We are gonna use the first one todazz

In [27]:
label_encoder = preprocessing.LabelEncoder()
X_cat=X_cat.apply(LabelEncoder().fit_transform)
X_cat

,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract
0,0,1,0,0,0,2,0,0,0,0,0
1,1,0,0,1,2,0,2,0,0,0,1
2,1,0,0,1,2,2,0,0,0,0,0
3,1,0,0,0,2,0,2,2,0,0,1
4,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7038,1,1,1,1,2,0,2,2,2,2,1
7039,0,1,1,1,0,2,2,0,2,2,1
7040,0,1,1,0,2,0,0,0,0,0,0
7041,1,1,0,1,0,0,0,0,0,0,0


In [28]:
X= pd.concat([X_num,X_cat], axis=1) # Our X is numerical only. Now we can apply SMOTE.
X

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract
0,0,1,29.85,29.85,0,1,0,0,0,2,0,0,0,0,0
1,0,34,56.95,1889.50,1,0,0,1,2,0,2,0,0,0,1
2,0,2,53.85,108.15,1,0,0,1,2,2,0,0,0,0,0
3,0,45,42.30,1840.75,1,0,0,0,2,0,2,2,0,0,1
4,0,2,70.70,151.65,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,24,84.80,1990.50,1,1,1,1,2,0,2,2,2,2,1
7039,0,72,103.20,7362.90,0,1,1,1,0,2,2,0,2,2,1
7040,0,11,29.60,346.45,0,1,1,0,2,0,0,0,0,0,0
7041,1,4,74.40,306.60,1,1,0,1,0,0,0,0,0,0,0


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
smote = SMOTE()
X_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

Churn
No       5163
Yes      5163
dtype: int64

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=100)

In [32]:
rfc = RandomForestClassifier()
rfc.fit(X_sm,y_sm)

RandomForestClassifier()

In [33]:
y_predictions=rfc.predict(X_test)
print(classification_report(y_test, y_predictions))

              precision    recall  f1-score   support

          No       1.00      1.00      1.00      1052
         Yes       1.00      1.00      1.00      1014

    accuracy                           1.00      2066
   macro avg       1.00      1.00      1.00      2066
weighted avg       1.00      1.00      1.00      2066

